In [1]:
import datetime as dt
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
%pip install yfinance
import yfinance as yf
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%pip install pandas-ta
import pandas_ta as ta
import time

from warnings import filterwarnings
# Raw Package
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
%pip install ta
from ta.trend import MACD, ADXIndicator
from ta.momentum import RSIIndicator
from ta.momentum import StochasticOscillator

# Market Data 
import yfinance as yf

#Graphing/Visualization
import datetime as dt 
import plotly
import plotly.graph_objs as go
import warnings

import pandas as pd
%pip install yahoo_fin
from yahoo_fin import stock_info as si

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 847 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 5.3 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218922 sha256=611f4bba4f65201686391137cc81520d1b9a50083b4552cb42ef0ec3b8b56a5e
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29104 sha256=84b108bed1a4194b8a9ede86dafa58dfae9d257bbdac5e5c32620a4d47b317c2
  Stored in directory: /root/.cache/pip/wheels/31/31/f1/f2ff471bbc5b84a4b973698ceecdd453ae043971791adc3431
Successfully built ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 4.2 MB/s 
     |████████████████

In [2]:
warnings.filterwarnings("ignore")

goodStocks = []
gStockFrames = []

def plotStock(df, macd, rsi, stock):
  # Declare plotly figure (go)
  fig=go.Figure()

  # add subplot properties when initializing fig variable
  fig = plotly.subplots.make_subplots(rows=6, cols=1, shared_xaxes=True,
                      vertical_spacing=0.01, 
                      row_heights=[0.5,0.1,0.2,0.2,0.2,0.2])

  fig.add_trace(go.Candlestick(x=df.index,
                  open=df['Open'],
                  high=df['High'],
                  low=df['Low'],
                  close=df['Close'], name = 'market data'))

  fig.add_trace(go.Scatter(x=df.index, 
                           y=df['MA5'], 
                           opacity=0.7, 
                           line=dict(color='blue', width=2), 
                           name='MA 5'))

  fig.add_trace(go.Scatter(x=df.index, 
                           y=df['MA20'], 
                           opacity=0.7, 
                           line=dict(color='orange', width=2), 
                           name='MA 20'))

  # Plot volume trace on 2nd row
  colors = ['green' if row['Open'] - row['Close'] >= 0 
            else 'red' for index, row in df.iterrows()]
  fig.add_trace(go.Bar(x=df.index, 
                       y=df['Volume'],
                       marker_color=colors
                      ), row=2, col=1)

  # Plot MACD trace on 3rd row
  colorsM = ['green' if val >= 0 
            else 'red' for val in macd.macd_diff()]
  fig.add_trace(go.Bar(x=df.index, 
                       y=macd.macd_diff(),
                       marker_color=colorsM
                      ), row=3, col=1)
  fig.add_trace(go.Scatter(x=df.index,
                           y=macd.macd(),
                           line=dict(color='black', width=2)
                          ), row=3, col=1)
  fig.add_trace(go.Scatter(x=df.index,
                           y=macd.macd_signal(),
                           line=dict(color='blue', width=1)
                          ), row=3, col=1)

  # update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
  fig.update_layout(height=900, width=1200, 
                    showlegend=False, 
                    xaxis_rangeslider_visible=False)


  # Make the title dynamic to reflect whichever stock we are analyzing
  fig.update_layout(
      title= str(stock)+' Live Share Price:',
      yaxis_title='Stock Price (USD per Shares)') 

  # update y-axis label
  fig.update_yaxes(title_text="Price", row=1, col=1)
  fig.update_yaxes(title_text="Volume", row=2, col=1)
  fig.update_yaxes(title_text="MACD", showgrid=False, row=3, col=1)
  fig.update_yaxes(title_text="Stoch", row=4, col=1)
  fig.update_yaxes(title_text="RSI", row=5, col=1)
  fig.update_yaxes(title_text="DMI", row=6, col=1)

  fig.update_xaxes(
      rangeslider_visible=False,
      rangeselector_visible=False,
      rangeselector=dict(
          buttons=list([
              dict(count=15, label="15m", step="minute", stepmode="backward"),
              dict(count=45, label="45m", step="minute", stepmode="backward"),
              dict(count=1, label="HTD", step="hour", stepmode="todate"),
              dict(count=1, label="1h", step="hour", stepmode="backward"),
              dict(count=4, label="4h", step="hour", stepmode="backward"),
              dict(count=1, label="1d", step="day", stepmode="backward"),
              dict(count=7, label="1wk", step="day", stepmode="backward"),
              dict(count=30, label="1mo", step="day", stepmode="backward"),
              dict(step="all")
          ])
      )
  )
  fig.show()
  time.sleep(2)
  plt.pause(0.005)

def scanStock(df):
  score = 0
  undervalued = False
  macdCross = False
  if(len(df) >= 39):
    if (df['Close'][len(df)-1] is not None and df['bookValue'][len(df)-1] is not None):
      if (df['Close'][len(df)-1] < df['bookValue'][len(df)-1]):
        undervalued = True
        score+=1
        print("UNDERVALUED");
    if(df['MACD'][len(df)-1] > df['MACD_sig'][len(df)-1]):
      macdCross = True
      score+=1
      print("MACD CROSSED")
  return score, macdCross

def checkStocks(stockList):
  # Override Yahoo Finance
  yf.pdr_override()

  # Create input field for our desired stock 
  #stock=input("Enter a stock ticker symbol: ")

  for i in range(len(stockList)):
    stock = stockList[i]
    print(stock)

    # Retrieve stock data frame (df) from yfinance API at an interval of 1m 
    df = yf.download(tickers=stock,period='5y',interval='1mo')

    getInfo = yf.Ticker(stock)

    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()

    if(len(df) >= 39):
      #ADX
      adx = ADXIndicator(df["High"], df["Low"], df["Close"], window=14, fillna=True)

      #RSI
      rsi = RSIIndicator(close=df['Close'], window=14, fillna=True)

      # MACD
      macd = MACD(close=df['Close'], 
                  window_slow=26,
                  window_fast=12, 
                  window_sign=9)
      
      df['RSI'] = rsi.rsi()
      df['MACD'] = macd.macd()
      df['MACD_diff'] = macd.macd_diff()
      df['MACD_sig'] = macd.macd_signal()
      if('bookValue' in getInfo.info):
        df['bookValue'] = getInfo.info['bookValue']
      else:
        df['bookValue'] = 0
    else:
      df['bookValue'] = 0

    thisStockScore, dmiCross = scanStock(df)

    if(dmiCross):
      print("\n \033[1m" + str(stock) + " Book Value: ")
      print(df['bookValue'])
      #goodStocks.append(stock)
      plotStock(df, macd, rsi, stock)

In [3]:



# gather stock symbols from major US exchanges
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

df1.reset_index

# convert DataFrame to list, then to sets
sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

# join the 4 sets into one. Because it's a set, there will be no duplicate symbols
symbols = set.union( sym1, sym2, sym3, sym4 )

# Some stocks are 5 characters. Those stocks with the suffixes listed below are not of interest.
my_list = ['W', 'R', 'P', 'Q']
del_set = set()
sav_set = set()

for symbol in symbols:
    if len( symbol ) > 4 and symbol[-1] in my_list:
        del_set.add( symbol )
    else:
        sav_set.add( symbol )

print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )
#goodStocks = checkStocks(stockList)
presetList=[]
#checkStocks(presetList)

#testList = df1.drop()
#checkStocks(presetList)
checkStocks(df1[0])
checkStocks(df2[0])
checkStocks(df3[0])





Removed 1093 unqualified stock symbols...
There are 11023 qualified stock symbols...
A
[*********************100%***********************]  1 of 1 completed
AAL
[*********************100%***********************]  1 of 1 completed
MACD CROSSED

 AAL Book Value: 
Date
2017-12-01   -12.146
2018-01-01   -12.146
2018-02-01   -12.146
2018-03-01   -12.146
2018-04-01   -12.146
               ...  
2022-08-01   -12.146
2022-09-01   -12.146
2022-10-01   -12.146
2022-11-01   -12.146
2022-11-14   -12.146
Name: bookValue, Length: 61, dtype: float64


AAP
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
ABBV
[*********************100%***********************]  1 of 1 completed
ABC
[*********************100%***********************]  1 of 1 completed
MACD CROSSED

 ABC Book Value: 
Date
2017-12-01    0.344
2018-01-01    0.344
2018-02-01    0.344
2018-03-01    0.344
2018-04-01    0.344
              ...  
2022-08-01    0.344
2022-09-01    0.344
2022-10-01    0.344
2022-11-01    0.344
2022-11-14    0.344
Name: bookValue, Length: 61, dtype: float64


ABMD
[*********************100%***********************]  1 of 1 completed
MACD CROSSED

 ABMD Book Value: 
Date
2017-12-01    34.087
2018-01-01    34.087
2018-02-01    34.087
2018-03-01    34.087
2018-04-01    34.087
               ...  
2022-08-01    34.087
2022-09-01    34.087
2022-10-01    34.087
2022-11-01    34.087
2022-11-14    34.087
Name: bookValue, Length: 61, dtype: float64


ABT
[*********************100%***********************]  1 of 1 completed
ACGL
[*********************100%***********************]  1 of 1 completed
MACD CROSSED

 ACGL Book Value: 
Date
2017-12-01    29.857
2018-01-01    29.857
2018-02-01    29.857
2018-03-01    29.857
2018-04-01    29.857
               ...  
2022-08-01    29.857
2022-09-01    29.857
2022-10-01    29.857
2022-11-01    29.857
2022-11-14    29.857
Name: bookValue, Length: 61, dtype: float64


ACN
[*********************100%***********************]  1 of 1 completed
ADBE
[*********************100%***********************]  1 of 1 completed
ADI
[*********************100%***********************]  1 of 1 completed


KeyboardInterrupt: ignored

In [ ]:
#print(df1[0])
#df1 = df1.drop([0])
print(df2[0][91])
#plotStock()